In [1]:
import numpy as np
import pandas as pd
import graph_tool as gt
from graph_tool import stats, inference, topology
import matplotlib.pyplot as plt
import itertools as it 
import pickle as pkl

In [2]:
co_offending_table = pd.read_csv('./raw_datasets/Cooffending.csv')
co_offending_table.drop_duplicates(inplace=True)

In [3]:
def build_co_offending_network_from_df(co_offending_df):
    offender_ids = np.unique(co_offending_df.NoUnique)
    ## Number of verticies is just the number of (unique) offenders
    N = len(offender_ids)
    g = gt.Graph(directed=False)
    g.add_vertex(N)
    no_unique = g.new_vertex_property('int')
    
    print('adding nodes')
    
    ## Add offenders as nodes and store a mapping from offender_ids
    ## to vertex index in the graph
    no_unique_to_index = {} 
    for (i, offender_id) in enumerate(offender_ids):
        no_unique[i] = offender_id
        no_unique_to_index[offender_id] = i

    g.vertex_properties['no_unique'] = no_unique
    
    print('nodes added')
    
    ## Add (unweighted) edges between offenders
    ## who are arrested together 
    
    print('adding edges')
    
    edge_iterators = co_offending_df.groupby('SeqE').apply(
        lambda x: it.combinations(map(lambda y: no_unique_to_index[y], x.NoUnique.values), 2))   
    for iterator in edge_iterators:
        for (source, dest) in iterator:
            # no self loops
            if source == dest:
                continue
            else:
                g.add_edge(source, dest)
                g.add_edge(dest, source)
    
    print('edges added')
    
    # may have added edges many times so we need to remove parallel edges
    gt.stats.remove_parallel_edges(g)
    
    return(g, no_unique_to_index)

In [4]:
def build_co_offender_network_from_year_range(start_year, end_year, co_offending_table):
    filtered_df = co_offending_table[(co_offending_table.annee >= start_year) & 
                                     (co_offending_table.annee < end_year)]
    res = build_co_offending_network_from_df(filtered_df)
    return(res)

## Generate Graphs (DO NOT RUN AGAIN - LOAD FROM MEMORY)

In [5]:
(g_train, mapping_train) = build_co_offender_network_from_year_range(2003, 2007, co_offending_table)
(g_test, mapping_test) = build_co_offender_network_from_year_range(2007, 2012, co_offending_table)

adding nodes
nodes added
adding edges
edges added
adding nodes
nodes added
adding edges
edges added


In [6]:
g_train.save('./2003_2007_coffender_graph.gml')
g_test.save('./2007_2012_cooffender_graph.gml')
pkl.dump( mapping_train, open( "./2003_2007_coffender_map.p", "wb" ) )
pkl.dump( mapping_test, open( "./2007_2012_coffender_map.p", "wb" ) )

## Load Graphs From Memory

In [2]:
#### LOAD FROM SAVED VALUES 
g_train = gt.Graph(directed=False)
g_test = gt.Graph(directed=False)
g_train.load(file_name="./2003_2007_coffender_graph.gml")
g_test.load(file_name="./2007_2012_cooffender_graph.gml")
mapping_train = pkl.load(open( "./2003_2007_coffender_map.p", "rb" ) )
mapping_test = pkl.load(open( "./2007_2012_coffender_map.p", "rb" ) )

In [3]:
g_train.set_vertex_filter(None)
g_test.set_vertex_filter(None)

In [4]:
g_train.num_vertices(), g_train.num_edges(), g_test.num_vertices(), g_test.num_edges()

(311065, 87280, 310434, 92879)

## APPROACH 1 - Simply Using the Connected Components as Communities

In [9]:
## Remove degree zero nodes from the graph 
degree_map_train = g_train.degree_property_map('out')
filter_map_train = g_train.new_vertex_property('bool')

for vertex in g_train.vertices():
    if degree_map_train[vertex] > 0:
        filter_map_train[vertex] = True
    else:
        filter_map_train[vertex] = False
        
g_train.set_vertex_filter(filter_map_train)

In [10]:
g_train.num_vertices(), g_train.num_edges()

(62968, 87280)

In [11]:
## Get components of training graph 

## Super Simple Approach - Just Use Connected Components 
component_map_train, hist = gt.topology.label_components(g_train)

In [12]:
## Use components from training graph to construct the test graph
degree_map_test = g_test.degree_property_map('out')
filter_map_test = g_test.new_vertex_property('bool')
component_map_test = g_test.new_vertex_property('int')

offenders_in_training_graph = set([g_train.vertex_properties['no_unique'][vertex] for 
                                   vertex in g_train.vertices()])

for vertex in g_test.vertices():
    co_offender_id = g_test.vertex_properties['no_unique'][vertex]
    if degree_map_test[vertex] > 0 and co_offender_id in offenders_in_training_graph:
        # if offender is in the training graph 
        # then we keep in the test graph 
        # and use the component label 
        # from the training graph 
        filter_map_test[vertex] = True
        vertex_index_in_training = mapping_train[co_offender_id]
        component_map_test[vertex] = component_map_train[vertex_index_in_training]
    else:
        filter_map_test[vertex] = False
        component_map_test[vertex] = -1

g_test.vertex_properties['component'] = component_map_test
g_test.set_vertex_filter(filter_map_test)

In [13]:
g_test.num_vertices(), g_test.num_edges()

(8417, 5737)

In [14]:
gt.inference.modularity(g_test, g_test.vertex_properties['component'])

0.2892617389714291

## APPROACH 2 - Analysis Restricted To The Largest Connected Component 

In [15]:
g_train.set_vertex_filter(None)
g_test.set_vertex_filter(None)

In [16]:
## Restrict the Training Set To the Largest Component 
largest_component_train = gt.topology.label_largest_component(g_train)
g_train.set_vertex_filter(largest_component_train)

In [17]:
g_train.num_vertices(), g_train.num_edges()

(1459, 6255)

In [ ]:
##### RUNS OUT OF MEMORY! :(

community_model_train = gt.inference.minimize_blockmodel_dl(g_train,B_max=None, B_min=None, verbose=True)

    B: 849 <- 1459    shrinking 1459 -> 1122
    B: 849 <- 1459    B=1122  niter:     1  count:    0  breaks:  0  min_S: 45327.534  max_S: 45338.414  S: 45327.534  ΔS:     -10.8802  moves:    18 
    B: 849 <- 1459    B=1122  niter:     2  count:    1  breaks:  1  min_S: 45327.534  max_S: 45338.414  S: 45325.942  ΔS:     -1.59256  moves:     6 
    B: 849 <- 1459    shrinking 1122 -> 863
    B: 849 <- 1459    B=863  niter:     1  count:    0  breaks:  0  min_S: 42757.071  max_S: 42793.480  S: 42757.071  ΔS:     -36.4090  moves:    43 
    B: 849 <- 1459    B=863  niter:     2  count:    0  breaks:  0  min_S: 42745.884  max_S: 42793.480  S: 42745.884  ΔS:     -11.1869  moves:    14 
    B: 849 <- 1459    B=863  niter:     3  count:    1  breaks:  1  min_S: 42745.884  max_S: 42793.480  S: 42743.111  ΔS:     -2.77236  moves:     7 
    B: 849 <- 1459    shrinking 863 -> 849
    B: 849 <- 1459    B=849  niter:     1  count:    1  breaks:  1  min_S: 42569.467  max_S: 42569.467  S: 42568.774

In [ ]:
## Use components from training graph to construct the test graph
degree_map_test = g_test.degree_property_map('out')
filter_map_test = g_test.new_vertex_property('bool')
component_map_test = g_test.new_vertex_property('int')

offenders_in_training_graph = set([g_train.vertex_properties['no_unique'][vertex] for 
                                   vertex in g_train.vertices()])

for vertex in g_test.vertices():
    co_offender_id = g_test.vertex_properties['no_unique'][vertex]
    if degree_map_test[vertex] > 0 and co_offender_id in offenders_in_training_graph:
        # if offender is in the training graph 
        # then we keep in the test graph 
        # and use the component label 
        # from the training graph 
        filter_map_test[vertex] = True
        vertex_index_in_training = mapping_train[co_offender_id]
        # Look up component in block model model 
        component_map_test[vertex] = community_model_train.b[vertex]
    else:
        filter_map_test[vertex] = False
        component_map_test[vertex] = -1

g_test.vertex_properties['component'] = component_map_test
g_test.set_vertex_filter(filter_map_test)

In [ ]:
gt.inference.modularity(g_test, g_test.vertex_properties['component'])